In [2]:
# Install, import libraries
import sys
!{sys.executable} -m pip install google-api-python-client earthengine-api

import geopandas as gpd
import pandas as pd
from pydap.client import open_url
from pydap.cas.urs import setup_session

import xarray as xr
import requests
import re

%load_ext dotenv
%dotenv
from os import environ

In [3]:
## Run this if you don't have a .nc file to work with

base_url = "https://oco2.gesdisc.eosdis.nasa.gov"
def list_urls(year):
    r = requests.get(f"https://oco2.gesdisc.eosdis.nasa.gov/opendap/OCO2_L2_Lite_FP.9r/{year}/catalog.xml")
    paths = re.findall("ID=\"(/opendap/hyrax/OCO2_L2_Lite_FP.9r/\d{4}/.*.nc4)\"", r.text)
    print(f"Found {len(paths)} files for {year}.")
    return paths

## Based on code from http://xarray.pydata.org/en/stable/io.html
def read_datasets(files, dim, transform_func=None):
    prog_ = 0
    session = setup_session(environ.get("EARTHDATA_USER"), environ.get("EARTHDATA_PASS"), check_url=base_url+files[0])
    def process_one_path(path,i):
        print(f"Loading dataset {i}/{len(files)}")
        # use a context manager, to ensure the file gets closed after use
        with xr.backends.PydapDataStore.open(path, session=session) as store:
            with xr.open_dataset(store) as ds:
                # transform_func should do some sort of selection or aggregation
                # aggregation
                if transform_func is not None:
                    ds = transform_func(ds)
                ds.load()
                return ds

    paths = sorted(files) # TODO use Glob
    datasets = [process_one_path(base_url+p,i) for i,p in enumerate(paths)]
    
    return xr.concat(datasets, dim)

# Method to reshape one dataset
def reshape_dataset(ds):
    return ds[[ 'sounding_id', 'latitude', 'longitude', 'time', 'xco2' ]]

# here we suppose we only care about the combined mean of each file;
# you might also use indexing operations like .sel to subset datasets
combined = read_datasets(list_urls(2015), dim='sounding_id', transform_func=reshape_dataset)

print(f"Done. Loaded {combined.nbytes * (2 ** -20)} MB into a dataset.")

combined.to_netcdf('2015_soundings.nc')

nc_file = combined.set_index(sounding_id=['latitude', 'longitude', 'time'])

Found 325 files for 2015.
Loading dataset 0/325
Loading dataset 1/325
Loading dataset 2/325
Loading dataset 3/325
Loading dataset 4/325
Loading dataset 5/325
Loading dataset 6/325
Loading dataset 7/325
Loading dataset 8/325
Loading dataset 9/325
Loading dataset 10/325
Loading dataset 11/325
Loading dataset 12/325
Loading dataset 13/325
Loading dataset 14/325
Loading dataset 15/325
Loading dataset 16/325
Loading dataset 17/325
Loading dataset 18/325
Loading dataset 19/325
Loading dataset 20/325
Loading dataset 21/325
Loading dataset 22/325
Loading dataset 23/325
Loading dataset 24/325
Loading dataset 25/325
Loading dataset 26/325
Loading dataset 27/325
Loading dataset 28/325
Loading dataset 29/325
Loading dataset 30/325
Loading dataset 31/325
Loading dataset 32/325
Loading dataset 33/325
Loading dataset 34/325
Loading dataset 35/325
Loading dataset 36/325
Loading dataset 37/325
Loading dataset 38/325
Loading dataset 39/325
Loading dataset 40/325
Loading dataset 41/325
Loading dataset 42

In [7]:
## Run this if you already have a file

nc_file = xr.open_dataset('2017_soundings_backup.nc')
nc_file

<xarray.Dataset>
Dimensions:      (sounding_id: 53034523)
Coordinates:
  * sounding_id  (sounding_id) int64 0 1 2 3 4 ... 204061 204062 204063 204064
Data variables:
    latitude     (sounding_id) float32 ...
    longitude    (sounding_id) float32 ...
    time         (sounding_id) datetime64[ns] ...
    xco2         (sounding_id) float32 ...
Attributes:
    identifier_product_doi:            10.5067/W8QGIYNKS3JC
    identifier_product_doi_authority:  http://dx.doi.org/
    filter_function:                   oco2_xco2_quality_flag_b9
    bc_function:                       oco2_bias_correct_b9
    warn_level_function:               oco2_warn_levels_b8
    Bias_Correction_land:              XCO2_Bias_Corrected = (XCO2_Raw + 0.90...
    Bias_Correction_oceanGL:           XCO2_Bias_Corrected = (XCO2_Raw + 0.24...
    Footprint_bias_land:               Assumed footprint biases in xco2 [ppm]...
    Footprint_bias_oceanGL:            Assumed footprint biases in xco2 [ppm]...
    Bias_Uncertainty:                  Bias correction parameter uncertaintie...
    Note_regarding_bias_correction:    There is some uncertainty in not only ...
    Platform:                          OCO-2
    Sensor:                            OCO-2
    title:                             ACOS L2 Lite Output
    BuildId:                           B8.1.00
    filtering_state:                   Filtered with oco2_lite_file_prefilter...
    creation_date:                     Sep 2018
    contact:                           Chris O'Dell: odell@atmos.colostate.edu

In [ ]:
pd_frame = nc_file.to_dataframe()
pd_frame['geometry'] = pd_frame.apply(lambda d: f"POINT ({float(d.longitude)} {d.latitude})", axis=1)
pd_frame = pd_frame.drop(columns['longitude', 'latitude', 'sounding_id'])
pd_frame

In [ ]:
gpd_frame = gpd.GeoDataFrame(pd_frame, geometry='geometry')
gpd_frame

In [ ]:
gpd_frame.to_file('oco2_out/2018/oco2_2018.shp', driver='ESRI Shapefile')